In [1]:
!pip install pygame==2.0.0.dev10
!pip install qiskit

In [2]:
## Adding Quantum To The Game Wormy

import random
import pygame
import sys
from pygame.locals import *

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, Aer

FPS = 15
WINDOWWIDTH = 640
WINDOWHEIGHT = 480
CELLSIZE = 20
assert WINDOWWIDTH % CELLSIZE == 0
assert WINDOWHEIGHT % CELLSIZE == 0
CELLWIDTH = int(WINDOWWIDTH / CELLSIZE)
CELLHEIGHT = int(WINDOWHEIGHT / CELLSIZE)

pygame 2.0.0.dev10 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
DARKGREEN = (0, 155, 0)
DARKGRAY = (40, 40, 40)
BGCOLOR = BLACK

UP = 'up'
DOWN = 'down'
LEFT = 'left'
RIGHT = 'right'

HEAD = 0 # the worm's head

In [4]:
def main():
    global FPSCLOCK, DISPLAYSURF, BASICFONT
    
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    BASICFONT = pygame.font.Font('freesansbold.ttf', 18)
    pygame.display.set_caption('Quantum Wormy')
    
    
    showStartScreen()
    while True:
        runGame()
        showGameOverScreen()

In [5]:
def runGame():
    
    device = 'qasm_simulator'
        
    simulator = Aer.get_backend(device)
    
    c = ClassicalRegister(1)
    q = QuantumRegister(1)
    circ = QuantumCircuit(q, c)
    circ.measure(q, c)
    circ.h(q[0])
    circ.measure(q, c)
        
    startx = random.randint(5, CELLWIDTH-6)
    starty = random.randint(5, CELLWIDTH-6)
    wormCoords = [{'x': startx, 'y': starty},
                 {'x': startx-1, 'y': starty},
                 {'x': startx-2, 'y': starty}]
    
    direction = RIGHT
    redApple = getRandomLocation() ## add in quantum things
    blueApple = getRandomLocation()
    
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                terminate()
            elif event.type == KEYDOWN:
                if (event.key == K_LEFT or event.key == K_a) and direction != RIGHT:
                    direction = LEFT
                elif (event.key == K_RIGHT or event.key == K_d) and direction != LEFT:
                    direction = RIGHT
                elif (event.key == K_UP or event.key == K_w) and direction != DOWN:
                    direction = UP
                elif (event.key == K_DOWN or event.key == K_s) and direction != UP:
                    direction = DOWN
                elif event.key == K_ESCAPE:
                    terminate()
                    

        if wormCoords[HEAD]['x'] == -1 or wormCoords[HEAD]['x']  == CELLWIDTH or wormCoords[HEAD]['y']== -1 or wormCoords[HEAD]['y']==CELLHEIGHT:
            return # gameover
        for wormBody in wormCoords[1:]:
            if wormBody['x'] == wormCoords[HEAD]['x'] and wormBody['y'] == wormCoords[HEAD]['y']:
                return # gameover
        
        if wormCoords[HEAD]['x'] == redApple['x'] and wormCoords[HEAD]['y'] == redApple['y']:
            redApple = getRandomLocation()
        elif wormCoords[HEAD]['x'] == blueApple['x'] and wormCoords[HEAD]['y'] == blueApple['y']:
            result = execute(circ, backend=simulator, shots=1).result()
            if result.get_counts(circ).get('0') is not None:
                for i in range(4):
                    if direction == UP:
                        new = {'x': wormCoords[-1]['x'], 'y': wormCoords[-1]['y']+i}
                    elif direction == DOWN:
                        new = {'x': wormCoords[-1]['x'], 'y': wormCoords[-1]['y']-i}
                    elif direction == LEFT:
                        new = {'x': wormCoords[-1]['x']+i, 'y': wormCoords[-1]['y']}
                    elif direction == RIGHT:
                        new = {'x': wormCoords[-1]['x']-i, 'y': wormCoords[-1]['y']}
                    wormCoords.insert(-1, new)
            else:
                if 5 > len(wormCoords):
                    return # game over
                for i in range(6):
                    del wormCoords[-1]    
            blueApple = getRandomLocation()
        else:
            del wormCoords[-1]
            
            
        if direction == UP:
            newHead = {'x': wormCoords[HEAD]['x'], 'y': wormCoords[HEAD]['y']-1}
        elif direction == DOWN:
            newHead = {'x': wormCoords[HEAD]['x'], 'y': wormCoords[HEAD]['y']+1}
        elif direction == LEFT:
            newHead = {'x': wormCoords[HEAD]['x']-1, 'y': wormCoords[HEAD]['y']}
        elif direction == RIGHT:
            newHead = {'x': wormCoords[HEAD]['x']+1, 'y': wormCoords[HEAD]['y']}
        wormCoords.insert(0, newHead)
        
        DISPLAYSURF.fill(BGCOLOR)
        drawGrid()
        drawWorm(wormCoords)
        drawApple(redApple, RED)
        drawApple(blueApple, BLUE)
        drawScore(len(wormCoords)-3)
        pygame.display.update()
        FPSCLOCK.tick(FPS)

In [6]:
def drawPressKeyMsg():
    pressKeySurf = BASICFONT.render('Press a key to play', True, DARKGRAY)
    pressKeyRect = pressKeySurf.get_rect()
    pressKeyRect.topleft = (WINDOWWIDTH-200, WINDOWHEIGHT-30)
    DISPLAYSURF.blit(pressKeySurf, pressKeyRect)

In [7]:
def checkForKeyPress():
    if len(pygame.event.get(QUIT))>0:
        terminate()
    
    keyUpEvents = pygame.event.get(KEYUP)
    if len(keyUpEvents)==0:
        return None
    if keyUpEvents[0].key == K_ESCAPE:
        terminate()
    return keyUpEvents[0].key

In [8]:
def showStartScreen():
    titleFont = pygame.font.Font('freesansbold.ttf', 50)
    titleSurf1 = titleFont.render('Quantum Wormy!', True, WHITE, DARKGREEN)
    titleSurf2 = titleFont.render('Quantum Wormy!', True, GREEN)
    
    degrees1 = 0
    degrees2 = 0
    while True:
        DISPLAYSURF.fill(BGCOLOR)
        
        rotatedSurf1 = pygame.transform.rotate(titleSurf1, degrees1)
        rotatedRect1 = rotatedSurf1.get_rect()
        rotatedRect1.center = (WINDOWWIDTH / 2, WINDOWHEIGHT / 2)
        DISPLAYSURF.blit(rotatedSurf1, rotatedRect1)
    
        rotatedSurf2 = pygame.transform.rotate(titleSurf2, degrees2)
        rotatedRect2 = rotatedSurf2.get_rect()
        rotatedRect2.center = (WINDOWWIDTH / 2, WINDOWHEIGHT / 2)
        DISPLAYSURF.blit(rotatedSurf2, rotatedRect2)
    
        drawPressKeyMsg()
    
        if checkForKeyPress():
            pygame.event.get()
            return
        pygame.display.update()
        FPSCLOCK.tick(FPS)
        
        degrees1 += 3
        degrees2 += 7

In [9]:
def terminate():
    pygame.quit()
    sys.exit()

In [10]:
def getRandomLocation():
    return {'x': random.randint(0, CELLWIDTH-1),'y': random.randint(0, CELLHEIGHT-1)}

In [11]:
def showGameOverScreen():
    gameOverFont = pygame.font.Font('freesansbold.ttf', 150)
    gameSurf = gameOverFont.render('Game', True, WHITE)
    overSurf = gameOverFont.render('Over', True, WHITE)
    gameRect = gameSurf.get_rect()
    overRect = overSurf.get_rect()
    gameRect.midtop = (WINDOWWIDTH / 2, 10)
    overRect.midtop = (WINDOWWIDTH / 2, gameRect.height + 10 + 25)
    
    DISPLAYSURF.blit(gameSurf, gameRect)
    DISPLAYSURF.blit(overSurf, overRect)
    drawPressKeyMsg()
    pygame.display.update()
    
    pygame.time.wait(500)
    checkForKeyPress()
    
    while True:
        if checkForKeyPress():
            pygame.event.get()
            return

In [12]:
def drawScore(score):
    scoreSurf = BASICFONT.render('Score: %s'%(score), True, WHITE)
    scoreRect = scoreSurf.get_rect()
    scoreRect.topleft = (WINDOWWIDTH-120, 10)
    DISPLAYSURF.blit(scoreSurf, scoreRect)

In [13]:
def drawWorm(wormCoords):
    for coord in wormCoords:
        x = coord['x'] * CELLSIZE
        y = coord['y'] * CELLSIZE
        wormSegmentRect = pygame.Rect(x, y, CELLSIZE, CELLSIZE)
        pygame.draw.rect(DISPLAYSURF, DARKGREEN, wormSegmentRect)
        wormInnerSegmentRect = pygame.Rect(x+4, y+4, CELLSIZE-8, CELLSIZE-8)
        pygame.draw.rect(DISPLAYSURF, GREEN, wormInnerSegmentRect)

In [14]:
def drawApple(coord, colour):
    x = coord['x'] * CELLSIZE
    y = coord['y'] * CELLSIZE
    appleRect = pygame.Rect(x, y, CELLSIZE, CELLSIZE)
    pygame.draw.rect(DISPLAYSURF, colour, appleRect)

In [15]:
def drawGrid():
    for x in range(0, WINDOWWIDTH, CELLSIZE):
        pygame.draw.line(DISPLAYSURF, DARKGRAY, (x, 0), (x, WINDOWHEIGHT))
    for y in range(0, WINDOWHEIGHT, CELLSIZE):
        pygame.draw.line(DISPLAYSURF, DARKGRAY, (0, y), (WINDOWWIDTH, y))

In [16]:
if __name__ == '__main__':
    main()

SystemExit: 

/home/alicia/dev/ENTER/envs/my-conda-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
